Web Scraping
---
Site: repetit.ru

City: Moscow

In [ ]:
pip install transliterate

In [ ]:
pip install tqdm

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from multiprocessing import Pool
from transliterate import translit, get_available_language_codes
import tqdm
from datetime import date

how many pages with 10 tutors on a page

In [ ]:
url_search = 'https://repetit.ru/repetitors/'
r_search = requests.get(url_search)
soup_search = BeautifulSoup(r_search.text, 'lxml')
max_search_pages = soup_search.find('div', {'class': 'pagination'}).find('p').get_text()[5:]
max_search_pages

'8870'

Scraping one page

In [ ]:
def parse_one_page(i):
    arr = []
    url_search_page = 'https://repetit.ru/repetitors/' + '?page='
    soup_search_page = BeautifulSoup(requests.get(url_search_page + str(i)).text, 'lxml')
    teacher = soup_search_page.find('div', {'class': 'teacher-card'})
    s = soup_search_page.find('script')
    if s:
        categories = re.findall(r'\'category\': \'.*\'', s.get_text())
        ids = re.findall(r'\'id\': \'\d*\'', s.get_text())
    else:
        categories, ids = np.nan, np.nan
    j = 0
    while teacher:
        arr.append([np.nan] * 13)

        try:
            # Tutor's categories
            arr[-1][0] = categories[j].replace("'category': '", '').replace("'", '').split(' - ')

            # Tutor's price 0-5000 rub per hour
            s = teacher.find('h3', {'class': 'price'}).get_text()[3:]
            arr[-1][1] = int(s.replace(' ', '').replace('от', '').replace('руб/час', ''))

            # Tutor's score 0.0-5.0
            s = str(teacher.find('div', {'class': 'star-rating'}))
            a = re.sub("[^0-9]", " ", s).split()
            arr[-1][2] = sum(map(int, a)) / 10

            # Tutor's working format
            arr[-1][3] = teacher.find('p', {'class': 'places-string'}).get_text().lower().split(', ')
            
            # Tutor's number of students' reviews
            s = teacher.find('a', {'class': 'show-all-reviews'}).get_text()
            arr[-1][4] = int(re.sub("[^0-9]", "", s))

            ###
            ### Adding this feature causes increasing of scraping time in hours
            ### due to requests to tutors' profiles. Search page doesn't contain
            ### information about the ages.
            ###
            # Tutor's age
            # a = ids[j].replace("'id': '", '').replace("'", '')
            # soup_tutor_page = BeautifulSoup(requests.get('https://repetit.ru/repetitor.aspx?id=' + a).text, 'lxml')
            # s = soup_tutor_page.find('div', {'class': 'row age'}).find('div', {'class': 'col-8'}).find('span', {'class': 'hide-in-edit-mode'}).contents
            # arr[-1][5] = int(re.sub("[^0-9]", "", str(s)))
            
            # Tutor's expirience in years
            s = teacher.find('span', {'class': 'stage'}).get_text()
            arr[-1][6] = int(re.sub("[^0-9]", "", s))

            # Tutor's status
            arr[-1][7] = teacher.find('p', {'class': 'status-stage'}).contents[0].strip()
            
            # Tutor's location
            s = teacher.find('div', {'class': 'places'})
            arr[-1][8] = s.get_text().strip().split('\n')

            # Tutor's list of tags
            arr[-1][9] = teacher.find('div', {'class': 'subjects-and-divisions'}).get_text().strip().split('\n')

            # Tutor's list of audience
            arr[-1][10] = teacher.find('div', {'class': 'categories'}).get_text().strip().split('\n')

            # Tutor's video presentation availability
            if teacher.find('div', {'class': 'video-presentation'}):
                arr[-1][11] = 'Yes'
            else:
                arr[-1][11] = 'No'
            
            # Tutor's photo availability
            s = str(teacher.find('img'))
            if 'no_photo' in s:
                arr[-1][12] = 'No'
            else:
                arr[-1][12] = 'Yes'
        except Exception:
            pass     

        teacher = teacher.find_next('div', {'class': 'teacher-card'})
        j += 1
    return arr

Main parser with multiprocessing

In [ ]:
def parse_search(url_search, max_search_pages):
    pages = [i for i in range(1, max_search_pages + 1)]
    pool = Pool(20)

    tutors_array_buff = []
    for result in tqdm.tqdm(pool.imap_unordered(parse_one_page, pages), total=len(pages)):
        tutors_array_buff.append(result)
    # print(tutors_array_buff)
    print()
    
    pool.close()
    pool.join()
    print('Web Scraping is completed')
    return tutors_array_buff

Web Scraping process

with default number of processe (2 cpus)

In [ ]:
# tutors_array_buff = parse_search(url_search, 20)
tutors_array_buff = parse_search(url_search, int(max_search_pages))

100%|██████████| 8870/8870 [1:25:03<00:00,  1.74it/s]


Web Scraping is completed


with 20 processes

In [ ]:
# tutors_array_buff = parse_search(url_search, 20)
tutors_array_buff = parse_search(url_search, int(max_search_pages))

100%|██████████| 8870/8870 [13:43<00:00, 10.78it/s]



Web Scraping is completed


Processing obtained data

In [ ]:
flat_list = [item for sublist in tutors_array_buff for item in sublist]

In [ ]:
tutors_array = np.array(flat_list)
tutors_array.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(88680, 13)

In [ ]:
columns = ['Categories',
           'Price',
           'Score',
           'Format',
           'Reviews_number',
           'Age',
           'Experience',
           'Status',
           'Location',
           'Tags',
           'Audience',
           'Video_presentation',
           'Photo']

In [ ]:
tutors = pd.DataFrame(tutors_array, columns=columns)
tutors.head()

,Categories,Price,Score,Format,Reviews_number,Age,Experience,Status,Location,Tags,Audience,Video_presentation,Photo
0,[Английский язык],2500,4.9,"[у репетитора, у ученика, дистанционно]",57,NaN,18,Частный преподаватель,[м. Авиамоторная],"[ОГЭ (ГИА), ЕГЭ, бизнес-курс, BEC, CAE, CPE, F...","[Дети 6-7 лет, Школьники 1-11 классов, Студент...",Yes,Yes
1,"[Физика, Математика]",2800,4.9,"[у репетитора, дистанционно]",52,NaN,19,Частный преподаватель,[г. Одинцово],"[ОГЭ (ГИА), ЕГЭ, подготовка к олимпиадам, шко...","[Школьники 5-11 классов, Студенты]",Yes,Yes
2,[Английский язык],4000,5,"[у репетитора, дистанционно]",50,NaN,38,Частный преподаватель,"[м. ВДНХ, м. Алексеевская, м. Фонвизинская, м....","[ОГЭ (ГИА), ЕГЭ, подготовка к олимпиадам, FCE,...","[Школьники 7-11 классов, Студенты]",Yes,Yes
3,"[Физика, Математика]",3500,4.9,"[у репетитора, у ученика, дистанционно]",38,NaN,8,Частный преподаватель,[м. Парк культуры (кольцевая)],"[ОГЭ (ГИА), ЕГЭ, подготовка к олимпиадам, шко...",[Школьники 6-11 классов],Yes,Yes
4,"[Английский язык, Русский как иностранный]",1500,5,"[у репетитора, у ученика, дистанционно]",32,NaN,17,Частный преподаватель,[м. Улица 1905 года],"[ОГЭ (ГИА), ЕГЭ, бизнес-курс, KET, PET, Общий ...","[Дети 6-7 лет, Школьники 1-11 классов, Студент...",No,Yes


In [ ]:
tutors.shape

(88680, 13)

with default number of processes

In [ ]:
tutors.isnull().sum()

Categories                0
Price                     0
Score                     0
Format                    0
Reviews_number        63188
Age                   88690
Experience            63188
Status                63188
Location              65414
Tags                  65427
Audience              65428
Video_presentation    65428
Photo                 65428
dtype: int64

with 20 processes

In [ ]:
tutors.isnull().sum()

Categories                0
Price                     0
Score                     0
Format                    0
Reviews_number        63164
Age                   88680
Experience            63164
Status                63164
Location              65387
Tags                  65400
Audience              65401
Video_presentation    65401
Photo                 65401
dtype: int64

In [ ]:
tutors_rus = tutors.copy()
tutors_eng = tutors.copy()

Translation and transliteration

Tags left without translation or transliteration

In [ ]:
translations_dict_ru_eng = {'Английский язык': 'English',
                            'Немецкий язык': 'German', 
                            'Итальянский язык': 'Italian',
                            'Французский язык': 'French',
                            'Русский как иностранный': 'Russian as a foreign language',
                            'Другой': 'Other',
                            'Русский язык': 'Russian',
                            'Экономика': 'Economics', 
                            'Математика': 'Mathematics',
                            'Обществознание': 'Social studies',
                            'Начальная школа': 'Elementary School', 
                            'География': 'Geography',
                            'Биология': 'Biology',
                            'Музыка': 'Music',
                            'Физика': 'Physics',
                            'Литература': 'Literature',
                            'История': 'History',
                            'Химия': 'Chemistry',
                            'Логопед': 'Speech therapist',
                            'Занятия с дошкольниками': 'Preschooler activities', 
                            'Информатика / программирование': 'Informatics / Programming',
                            'Испанский язык': 'Spanish', 
                            'Редкие иностранные языки': 'Rare foreign languages',
                            'Изобразительное искусство': 'Art',
                            'Китайский язык': 'Chinese',
                            'Спорт и фитнес': 'Sport and fitness',
                            'Японский язык': 'Japanese',
                            'у ученика': 'at the student\'s',
                            'у репетитора': 'at the tutor\'s',
                            'дистанционно': 'remotely',
                            'Частный преподаватель': 'Private tutor',
                            'Студент': 'Student',
                            'Аспирант': 'Postgraduate student',
                            'Школьный преподаватель': 'School teacher',
                            'Преподаватель вуза': 'University professor',
                            'Носитель языка': 'Native speaker',
                            'Школьники': 'Pupils of',
                            'классов': 'grades',
                            'класса': 'grade',
                            'Дети': 'Children',
                            'лет': 'years old',
                            'Студенты': 'Students',
                            'Взрослые': 'Adults',
                            'ы': 's'}

In [ ]:
cols_to_translate = ['Categories', 'Format', 'Status', 'Location', 'Tags', 'Audience']

In [ ]:
for element in translations_dict_ru_eng:
    for col in cols_to_translate:
        tutors_eng[col] = tutors_eng[col].apply(lambda x: str(x).replace(element, translations_dict_ru_eng[element]))

In [ ]:
tutors_eng.Location = tutors_eng.Location.apply(lambda x: translit(str(x), 'ru', reversed=True))
tutors_eng.Location = tutors_eng.Location.apply(lambda x: str(x).replace('m.', 'metro'))
tutors_eng.Location = tutors_eng.Location.apply(lambda x: str(x).replace('g.', 'city'))

In [ ]:
tutors_rus = tutors_rus.drop(['Age'], axis=1)
tutors_eng = tutors_eng.drop(['Age'], axis=1)

In [ ]:
for col in tutors_eng.columns:
    tutors_eng[col].replace('None', np.nan, inplace=True)

Checking None/Nan objects

In [ ]:
tutors_eng.isnull().sum()

Categories                0
Price                     0
Score                     0
Format                    0
Reviews_number        63174
Experience            63174
Status                63174
Location              65406
Tags                  65419
Audience              65420
Video_presentation    65420
Photo                 65420
dtype: int64

In [ ]:
for col in tutors_rus.columns:
    tutors_rus[col].replace('None', np.nan, inplace=True)

In [ ]:
tutors_rus.isnull().sum()

Categories                0
Price                     0
Score                     0
Format                    0
Reviews_number        63174
Experience            63174
Status                63174
Location              65406
Tags                  65419
Audience              65420
Video_presentation    65420
Photo                 65420
dtype: int64

In [ ]:
today = date.today()
d1 = today.strftime("%Y_%m_%d")

Save into csv

In [ ]:
tutors_rus.to_csv('tutors_rus_' + d1 + '.csv', index=False, encoding='utf-8-sig')
tutors_eng.to_csv('tutors_eng_' + d1 + '.csv', index=False, encoding='utf-8-sig')

In [ ]:
tutors_rus_read = pd.read_csv('tutors_rus_2021_10_06.csv')
tutors_eng_read = pd.read_csv('tutors_eng_2021_10_06.csv')

In [ ]:
tutors_rus_read.head()

,Categories,Price,Score,Format,Reviews_number,Experience,Status,Location,Tags,Audience,Video_presentation,Photo
0,"['Немецкий язык', 'Испанский язык']",1800,5.0,['дистанционно'],26.0,21.0,Частный преподаватель,NaN,NaN,NaN,NaN,NaN
1,['Математика'],2500,4.9,"['у репетитора', 'дистанционно']",41.0,29.0,Частный преподаватель,['м. Щукинская'],"['ОГЭ (ГИА)', 'ЕГЭ', 'подготовка к олимпиадам'...","['Школьники 4-11 классов', 'Студенты']",No,Yes
2,['Английский язык'],1500,5.0,"['у репетитора', 'у ученика', 'дистанционно']",14.0,11.0,Частный преподаватель,['м. Чертановская'],"['ОГЭ (ГИА)', 'ЕГЭ', 'подготовка к олимпиадам'...","['Дети 6-7 лет', 'Школьники 1-11 классов', 'Ст...",Yes,Yes
3,['Химия'],1300,5.0,['дистанционно'],41.0,39.0,Частный преподаватель,NaN,NaN,NaN,NaN,NaN
4,['Математика'],1500,5.0,"['у репетитора', 'у ученика', 'дистанционно']",35.0,9.0,Школьный преподаватель,"['м. Отрадное', 'м. Бабушкинская', 'м. Свиблов...","['ОГЭ (ГИА)', 'ЕГЭ', ' школьный курс', 'Алгебр...",['Школьники 5-11 классов'],No,Yes


In [ ]:
tutors_eng_read.head()

,Categories,Price,Score,Format,Reviews_number,Experience,Status,Location,Tags,Audience,Video_presentation,Photo
0,"['German', 'Spanish']",1800,5.0,['remotely'],26.0,21.0,Private tutor,NaN,NaN,NaN,NaN,NaN
1,['Mathematics'],2500,4.9,"['at the tutor's', 'remotely']",41.0,29.0,Private tutor,['metro Schukinskaja'],"['ОГЭ (ГИА)', 'ЕГЭ', 'подготовка к олимпиадам'...","['Pupils of 4-11 grades', 'Studentы']",No,Yes
2,['English'],1500,5.0,"['at the tutor's', 'at the student's', 'remote...",14.0,11.0,Private tutor,['metro Chertanovskaja'],"['ОГЭ (ГИА)', 'ЕГЭ', 'подготовка к олимпиадам'...","['Children 6-7 years old', 'Pupils of 1-11 gra...",Yes,Yes
3,['Chemistry'],1300,5.0,['remotely'],41.0,39.0,Private tutor,NaN,NaN,NaN,NaN,NaN
4,['Mathematics'],1500,5.0,"['at the tutor's', 'at the student's', 'remote...",35.0,9.0,School teacher,"['metro Otradnoe', 'metro Babushkinskaja', 'me...","['ОГЭ (ГИА)', 'ЕГЭ', ' школьный курс', 'Алгебр...",['Pupils of 5-11 grades'],No,Yes
